In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import onnxruntime as ort

In [15]:
# 1. 線形回帰モデルの定義
class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)  # 線形層

    def forward(self, x):
        return self.linear(x)

# データセットを生成 (y = 2.0 * x + 3.0 + noise)
torch.manual_seed(42)
X = torch.rand(100, 1) * 10  # 入力データ (100個のサンプル、1次元)
Y = 2.0 * X + 3.0 + torch.randn(100, 1) * 2  # 出力データ (ノイズ付き)

In [16]:
# 2. モデルのインスタンス化
input_dim = 1
output_dim = 1
model = LinearRegressionModel(input_dim, output_dim)

# 損失関数と最適化関数
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [17]:
# 3. モデルのトレーニング
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()

    # 順伝播
    outputs = model(X)
    loss = criterion(outputs, Y)

    # 逆伝播とパラメータ更新
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# トレーニング後のパラメータ
print(f'学習した重み: {model.linear.weight.item():.4f}')
print(f'学習したバイアス: {model.linear.bias.item():.4f}')


Epoch [100/1000], Loss: 3.7373
Epoch [200/1000], Loss: 3.0689
Epoch [300/1000], Loss: 2.8125
Epoch [400/1000], Loss: 2.7140
Epoch [500/1000], Loss: 2.6763
Epoch [600/1000], Loss: 2.6618
Epoch [700/1000], Loss: 2.6562
Epoch [800/1000], Loss: 2.6541
Epoch [900/1000], Loss: 2.6533
Epoch [1000/1000], Loss: 2.6529
学習した重み: 1.9118
学習したバイアス: 3.4333


In [18]:
# 4. ONNX形式に変換 (バッチサイズを柔軟にする)
torch.onnx.export(
    model,
    dummy_input,
    onnx_path,
    export_params=True,
    opset_version=11,
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}}
)
print(f"ONNX形式で保存しました: {onnx_path}")


ONNX形式で保存しました: linear_regression.onnx


In [19]:
# 5. ONNXで推論
ort_session = ort.InferenceSession(onnx_path)

# テストデータで推論
test_input = torch.tensor([[4.5], [7.0], [2.5]])  # テスト用入力
onnx_input = test_input.numpy().astype(np.float32)
onnx_outputs = ort_session.run(None, {"input": onnx_input})

print("ONNX推論結果:")
for inp, outp in zip(test_input, onnx_outputs[0]):
    print(f"入力: {inp.item():.2f}, 出力: {outp[0]:.2f}")

ONNX推論結果:
入力: 4.50, 出力: 12.04
入力: 7.00, 出力: 16.82
入力: 2.50, 出力: 8.21
